# Aufgabe 1

Implementieren Sie die LU -Zerlegung für Tridiagonal-Matrizen aus Aufgabe (3)(a) in Python und testen Sie Ihre Funktion an der Tridiagonalmatrix mit $bi = ci+1 = −i$ für $i = 1 ...,n −1$ und $ai = 4$
für alle $i = 1,...,n, n = 8$

In [8]:
import numpy as np 
import matplotlib.pyplot as plt 

In [9]:
# Make the Tridiagonal Matrix for Testing

def create_tridiagonal_matrix(n):
    a = 4 * np.ones(n)
    b = -np.arange(1, n)
    c = -np.arange(1, n)
    
    matrix = np.zeros((n, n))
    
    for i in range(n):
        # a on the diagonal
        matrix[i, i] = a[i]
        if i > 0:
            # b on the lower diagonal
            matrix[i, i-1] = b[i-1]
        if i < n-1:
            # c on the upper diagonal
            matrix[i, i+1] = c[i]
    
    return matrix

n = 8
T = create_tridiagonal_matrix(n)
print(T)


[[ 4. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  4. -2.  0.  0.  0.  0.  0.]
 [ 0. -2.  4. -3.  0.  0.  0.  0.]
 [ 0.  0. -3.  4. -4.  0.  0.  0.]
 [ 0.  0.  0. -4.  4. -5.  0.  0.]
 [ 0.  0.  0.  0. -5.  4. -6.  0.]
 [ 0.  0.  0.  0.  0. -6.  4. -7.]
 [ 0.  0.  0.  0.  0.  0. -7.  4.]]


Goal is to calculate the matrix equation $Ax = b$. For this we decompose A into

$PA = LU$ where $L$ is a lower triangular matrix and $U$ is a upper triagonal matrix, $P$ is the permutation matrix

### Use Aufgabe 3 as an Algorithm

So assume the Tridiagonal Matrix $T$ has a $LU$ Decompositon then


$L = \begin{pmatrix}1 & 0 & ...  & 0 \\
                    \gamma_1 & 1 & 0 & ... \\
                    0 & \gamma_3 & 1 & ... \\ \end{pmatrix}$

and 

$U = \begin{pmatrix} \alpha_1 & \beta_1 & 0 & ... & 0 \\
                    0 & \alpha_2 & \beta_2 & ... & 0 \\
                    0 & ... & ... &  \beta_{n-1} \\
                    0 & ... & ... & ... & \alpha_n \\ \end{pmatrix}$

we can recursivly calculate these coefficients

$\alpha_1 = a_1$

$\gamma_i = \frac{c_1}{\alpha_{i-1}}, \alpha_i = a_i - \gamma_i \beta_{i-1}, \beta_{i-1} = b_{i-1}$ for $i = 2...n$

In [10]:
# Small helper function
def extract_tridiagonal_elements(T):
    n = T.shape[0]
    a = np.diag(T)
    b = np.diag(T, k = 1)
    c = np.diag(T, k=-1)
    return a,b,c

extract_tridiagonal_elements(T)

(array([4., 4., 4., 4., 4., 4., 4., 4.]),
 array([-1., -2., -3., -4., -5., -6., -7.]),
 array([-1., -2., -3., -4., -5., -6., -7.]))

In [11]:
def lu_decomposition_tridiagonal(T):
    a,b,c = extract_tridiagonal_elements(T)
    n = len(a)
    alpha = np.zeros(n)
    beta = np.zeros(n-1)
    gamma = np.zeros(n-1)

    # Initialize the first alpha vlue
    alpha[0] = a[0]
    for i in range(1,n):
        gamma[i-1] = c[i-1] / alpha[i-1]
        alpha[i] = a[i] - gamma[i-1] * b[i-1]
        beta[i-1] = b[i-1]
    # Then we can create the L and U matrices
    L = np.eye(n)
    U = np.zeros((n,n))
    for i in range(n):
        U[i,i] = alpha[i]
        if i < n-1:
            L[i+1,i] = gamma[i]
            U[i,i+1] = beta[i]
    return L,U

In [12]:
# Perform LU decomposition using the custom function
L, U = lu_decomposition_tridiagonal(T)

print(L)
print(U)



[[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [-0.25        1.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -0.53333333  1.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.         -1.02272727  1.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.         -4.29268293  1.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.37962963  1.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.         -1.01726845
   1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   3.32761194  1.        ]]
[[  4.          -1.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           3.75        -2.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           2.93333333  -3.

In [13]:
# Check if we get the right matrix

LU = L @ U
print(LU)

np.allclose(T,LU)

[[ 4. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  4. -2.  0.  0.  0.  0.  0.]
 [ 0. -2.  4. -3.  0.  0.  0.  0.]
 [ 0.  0. -3.  4. -4.  0.  0.  0.]
 [ 0.  0.  0. -4.  4. -5.  0.  0.]
 [ 0.  0.  0.  0. -5.  4. -6.  0.]
 [ 0.  0.  0.  0.  0. -6.  4. -7.]
 [ 0.  0.  0.  0.  0.  0. -7.  4.]]


True

# Aufgabe 2



In [14]:
def count_operations(n, m, k):
    additions = 0
    multiplications = 0

    # Matrix C has n Rows and M collumns 
    for i in range(n):
        for j in range(k):
            # Each element of C gets calculates with the scalar product
            for l in range(m):
                multiplications += 1  # One Multiplikation with each element
                if l < m - 1:
                    additions += 1  # m-1 Addditions

    return additions, multiplications


n = 3
m = 2
k = 4

additions, multiplications = count_operations(n, m, k)
print(f"Anzahl der Additionen: {additions}")
print(f"Anzahl der Multiplikationen: {multiplications}")

Anzahl der Additionen: 12
Anzahl der Multiplikationen: 24
